In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_table("all_table_definitions.txt", delimiter="|", encoding= 'unicode_escape')

In [3]:
df.rename(columns=lambda x: x.strip(), inplace=True)

In [4]:
new = df[df['Field Name'].str.contains('Table     :')]['Field Name'].str.split(' ', expand=True)[6]
index= new.index.values
df1= df.copy()
df1['Table Name'] = np.NaN
df1.loc[index, ['Table Name']] = new.values
df1['Table Name'].fillna(method='ffill', inplace=True)
df1.head()

,Field Name,Description,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,Table Name
0,,...,,length,,,Table Mode,NaN
1,---------------+------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 indt,...,tcdate,UTC Date/T 4,No,Yes,Not applicab,NaN
3,2 dsca,...,tcmcs.str50m,Multibyte 200,No,Yes,Not applicab,NaN
4,3 wfrc,...,asint.wfld,0,No,Yes,Not applicab,NaN


In [5]:
new1 = df1[df1['Field Name'].str.contains('Delete Reference Message')]['Field Name'].str.split(' ', expand=True)[8]
index1= new.index.values
df2= df1.copy()
df2['VCR'] = np.NaN
df2.loc[index1, ['VCR']] = new1.values
df2['VCR'].fillna(method='ffill', inplace=True)
df3 = df2[df2['VCR'] == '107U0']
df3.head()

,Field Name,Description,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,Table Name,VCR
22,Table : asint000 ...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
23,VRC : 107U0 a000 (Update) ...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
24,...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
25,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
26,Field Name,Description ...,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,asint000,107U0


In [6]:
table_name = ['tccom001', 'tcmcs002', 'tcmcs010', 'tcmcs015', 'tcmcs023', 'tiedm010', 'tiedm100', 'tiedm110']

In [7]:
appended_data = []
table_data = []
for name in table_name:
    try:
        newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
        list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
        newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
        newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
        nan_value = float("NaN")
        newdf1.replace("", nan_value, inplace=True)
        newdf1.dropna(subset = ["Field Name"], inplace=True)
        newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
        newdf1['Datatype'] = newdf1['Datatype     Physic.'].str.split(' ',  expand=True, n=1)[1].str.split('  ',  expand=True, n=1)[0]
        newdf1['Physic.'] = newdf1['Datatype     Physic.'].str.split(' ',  expand=True, n=1)[1].str.split('  ',  expand=True, n=1)[1]
        appended_data.append(newdf1)
        table_data.append(name)
    except ValueError: 
        print("A ValueError occurred")
    except:
        print(name, "does not have any information" )
appended_data = pd.concat(appended_data)
appended_data.drop('Datatype     Physic.', axis=1, inplace=True)
print(table_data, "does have information")
appended_data

tcmcs015 does not have any information
['tccom001', 'tcmcs002', 'tcmcs010', 'tcmcs023', 'tiedm010', 'tiedm100', 'tiedm110'] does have information


,Field Name,Description,Domain,Mand.,Acti,Refer. Reference,Table Name,VCR,Datatype,Physic.
111763,emno,Employee,tcemno,Yes,Yes,Not applicab,tccom001,107U0,String,9
111764,nama,Name,tcnama,No,Yes,Not applicab,tccom001,107U0,Multibyte,140
111765,namb,Given Name,tccom.name,No,Yes,Not applicab,tccom001,107U0,Multibyte,200
111766,namc,Middle Name,tccom.name,No,Yes,Not applicab,tccom001,107U0,Multibyte,200
111767,namd,Last Name,tccom.name,No,Yes,Not applicab,tccom001,107U0,Multibyte,200
...,...,...,...,...,...,...,...,...,...,...
293570,usup,Use Up Materials Present,tcyesno,Yes,Yes,Not applicab,tiedm110,107U0,Enumerated,1
293571,rdsp,Reference Designators Present,tcyesno,Yes,Yes,Not applicab,tiedm110,107U0,Enumerated,1
293572,cmba,Combined E-Item & Revision & Positions,,,,Not applicab,tiedm110,107U0,,
293576,cmbb,Combined of E-Item & Revision,,,,tiedm100 Mandatory,tiedm110,107U0,,


In [8]:
appended_data.to_excel('test.xlsx')

In [9]:
databs = appended_data[['Field Name', 'Description','Table Name']]
databs['Field Name'] = 'T_' + databs['Field Name'].astype(str).str.upper()
newdata = databs[databs['Table Name'] == 'tcmcs002']
newdata

,Field Name,Description,Table Name
133973,T_CCUR,Currency,tcmcs002
133974,T_DSCA,Description,tcmcs002
133975,T_DSCB,Short Description,tcmcs002
133976,T_CRND,Rounding Factor,tcmcs002
133977,T_EMUC,EMU Currency,tcmcs002
133978,T_BREP,BLWI Reporting Currency,tcmcs002
133979,T_GTRF,Grand Total Rounding Factor,tcmcs002
133980,T_ICCC,ISO Currency Code,tcmcs002


In [10]:
table = pd.read_excel('ttcmcs002100.xlsx', header=1)
columns = list(newdata['Field Name'])
datset = table[columns]
datset

,T_CCUR,T_DSCA,T_DSCB,T_CRND,T_EMUC,T_BREP,T_GTRF,T_ICCC
0,JPY,Japanese Yen,JPY,0.0001,2.0,2.0,0.001,JPY
1,EUR,Euro,EUR,0.0001,2.0,2.0,0.001,EUR
2,USD,US Dollars,USD,0.0001,2.0,2.0,0.001,USD
3,PLN,Polish Zloty,PLN,0.0001,1.0,2.0,0.001,PLN
4,SEK,Swedish Kronor,SEK,0.0001,1.0,2.0,0.001,SEK
5,GBP,Pound Sterling,GBP,0.0001,1.0,2.0,0.001,GBP


In [11]:
engine = create_engine('sqlite:///database.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "tcmcs002"
datset.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

2022-09-08 09:56:03,246 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tcmcs002")
2022-09-08 09:56:03,248 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-08 09:56:03,249 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tcmcs002")
2022-09-08 09:56:03,249 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-08 09:56:03,251 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-09-08 09:56:03,255 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-08 09:56:03,258 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("tcmcs002")
2022-09-08 09:56:03,260 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-08 09:56:03,263 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-09-08 09:56:03,264 INFO sqlalchemy.engine.Engine [raw sql] ('tcmcs002',)
2022-09-08 09:56:03,266 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("tcmc

In [12]:
newdata['Name'] =  + newdata['Field Name'] + ' AS \'' + newdata['Field Name'] + ' (' + newdata['Description'] + ")'"
names = list(newdata['Name'])
new_lst=(','.join(names))
newdata

,Field Name,Description,Table Name,Name
133973,T_CCUR,Currency,tcmcs002,T_CCUR AS 'T_CCUR (Currency)'
133974,T_DSCA,Description,tcmcs002,T_DSCA AS 'T_DSCA (Description)'
133975,T_DSCB,Short Description,tcmcs002,T_DSCB AS 'T_DSCB (Short Description)'
133976,T_CRND,Rounding Factor,tcmcs002,T_CRND AS 'T_CRND (Rounding Factor)'
133977,T_EMUC,EMU Currency,tcmcs002,T_EMUC AS 'T_EMUC (EMU Currency)'
133978,T_BREP,BLWI Reporting Currency,tcmcs002,T_BREP AS 'T_BREP (BLWI Reporting Currency)'
133979,T_GTRF,Grand Total Rounding Factor,tcmcs002,T_GTRF AS 'T_GTRF (Grand Total Rounding Factor)'
133980,T_ICCC,ISO Currency Code,tcmcs002,T_ICCC AS 'T_ICCC (ISO Currency Code)'


In [13]:
query = """
DROP VIEW TESTtt"""
cursor.execute(query)
query = """
CREATE VIEW [TESTtt] AS 
SELECT  {}
FROM tcmcs002 """.format(new_lst)
cursor.execute(query)
cursor.fetchall()

[]

In [14]:
query = """
SELECT * FROM TESTtt  """
cursor.execute(query)
cursor.fetchall()

[('JPY', 'Japanese Yen', 'JPY', 0.0001, 2.0, 2.0, 0.001, 'JPY'),
 ('EUR', 'Euro', 'EUR', 0.0001, 2.0, 2.0, 0.001, 'EUR'),
 ('USD', 'US Dollars', 'USD', 0.0001, 2.0, 2.0, 0.001, 'USD'),
 ('PLN', 'Polish Zloty', 'PLN', 0.0001, 1.0, 2.0, 0.001, 'PLN'),
 ('SEK', 'Swedish Kronor', 'SEK', 0.0001, 1.0, 2.0, 0.001, 'SEK'),
 ('GBP', 'Pound Sterling', 'GBP', 0.0001, 1.0, 2.0, 0.001, 'GBP')]

In [15]:
data=cursor.execute('''SELECT * FROM TESTtt''')
for column in data.description:
    print(column[0])
conn.close()


T_CCUR (Currency)
T_DSCA (Description)
T_DSCB (Short Description)
T_CRND (Rounding Factor)
T_EMUC (EMU Currency)
T_BREP (BLWI Reporting Currency)
T_GTRF (Grand Total Rounding Factor)
T_ICCC (ISO Currency Code)
